In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
import torch
from pathlib import Path
import json
import PIL
from PIL import ImageDraw, ImageFont,Image, ImageOps, ImageEnhance
from matplotlib import patches, patheffects
import numpy as np
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from functools import partial
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms,models
from torchvision.utils import make_grid
import math
import random
import numbers
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm, tqdm_notebook
import glob
from skimage import io
from torch.autograd import Variable as V
from scipy import ndimage
import time
import os
import copy
import skimage
torch.backends.cudnn.benchmark=True
torch.cuda.set_device(0)
import scipy
def sum_geom(a,r,n): return a*n if r==1 else math.ceil(a*(1-r**n)/(1-r))
def is_listy(x): return isinstance(x, (list,tuple))
def is_iter(x): return isinstance(x, collections.Iterable)
def map_over(x, f): return [f(o) for o in x] if is_listy(x) else f(x)
def map_none(x, f): return None if x is None else f(x)
def delistify(x): return x[0] if is_listy(x) else x
def listify(x, y):
    if not is_iter(x): x=[x]
    n = y if type(y)==int else len(y)
    if len(x)==1: x = x * n
    return x

def datafy(x):
    if is_listy(x): return [o.data for o in x]
    else:           return x.data

conv_dict = {np.dtype('int8'): torch.LongTensor, np.dtype('int16'): torch.LongTensor,
    np.dtype('int32'): torch.LongTensor, np.dtype('int64'): torch.LongTensor,
    np.dtype('float32'): torch.FloatTensor, np.dtype('float64'): torch.FloatTensor}

def A(*a):
    """convert iterable object into numpy array"""
    return np.array(a[0]) if len(a)==1 else [np.array(o) for o in a]

def T(a, half=False, cuda=True):
    """
    Convert numpy array into a pytorch tensor. 
    if Cuda is available and USE_GPU=True, store resulting tensor in GPU.
    """
    if not torch.is_tensor(a):
        a = np.array(np.ascontiguousarray(a))
        if a.dtype in (np.int8, np.int16, np.int32, np.int64):
            a = torch.LongTensor(a.astype(np.int64))
        elif a.dtype in (np.float32, np.float64):
            a = to_half(a) if half else torch.FloatTensor(a)
        else: raise NotImplementedError(a.dtype)
    if cuda: a = to_gpu(a)
    return a

def to_half(tensor):
    if torch.cuda.is_available():
        return torch.cuda.HalfTensor(tensor)
    else:
        return torch.FloatTensor(tensor)

def create_variable(x, volatile, requires_grad=False):
    if type (x) != Variable:
        x = Variable(T(x), requires_grad=requires_grad, volatile=volatile)
    return x

def V_(x, requires_grad=False, volatile=False):
    '''equivalent to create_variable, which creates a pytorch tensor'''
    return create_variable(x, volatile=volatile, requires_grad=requires_grad)
def V(x, requires_grad=False, volatile=False):
    '''creates a single or a list of pytorch tensors, depending on input x. '''
    return map_over(x, lambda o: V_(o, requires_grad, volatile))

def VV_(x): 
    '''creates a volatile tensor, which does not require gradients. '''
    return create_variable(x, True)

def VV(x):
    '''creates a single or a list of pytorch tensors, depending on input x. '''
    return map_over(x, VV_)

def to_np(v):
    '''returns an np.array object given an input of np.array, list, tuple, torch variable or tensor.'''
    if isinstance(v, float): return np.array(v)
    if isinstance(v, (np.ndarray, np.generic)): return v
    if isinstance(v, (list,tuple)): return [to_np(o) for o in v]
    if isinstance(v, Variable): v=v.data
    if torch.cuda.is_available():
        if is_half_tensor(v): v=v.float()
    if isinstance(v, torch.FloatTensor): v=v.float()
    return v.cpu().numpy()

def is_half_tensor(v):
    return isinstance(v, torch.cuda.HalfTensor)


USE_GPU = torch.cuda.is_available()
def to_gpu(x, *args, **kwargs):
    '''puts pytorch variable to gpu, if cuda is available and USE_GPU is set to true. '''
    return x.cuda(*args, **kwargs) if USE_GPU else x

def noop(*args, **kwargs): return

def trainable_params_(m):
    '''Returns a list of trainable parameters in the model m. (i.e., those that require gradients.)'''
    return [p for p in m.parameters() if p.requires_grad]

def chain_params(p):
    if is_listy(p):
        return list(chain(*[trainable_params_(o) for o in p]))
    return trainable_params_(p)

def set_trainable_attr(m,b):
    m.trainable=b
    for p in m.parameters(): p.requires_grad=b

def apply_leaf(m, f):
    c = children(m)
    if isinstance(m, nn.Module): f(m)
    if len(c)>0:
        for l in c: apply_leaf(l,f)
def children(m): return m if isinstance(m, (list, tuple)) else list(m.children())
def set_trainable(l, b):
    apply_leaf(l, lambda m: set_trainable_attr(m,b))


In [ ]:
PATH = Path('Data/')

In [ ]:
Filenametoid = json.load((PATH/'filename_id.json').open())

In [ ]:
def GetIndex(n, cv_idx=0, val_pct=0.2, seed=42):
    n=n-1
    np.random.seed(seed)
    n_val = int(val_pct*n)
    idx_start = cv_idx*n_val
    idxs = np.random.permutation(n)
    val=idxs[idx_start:idx_start+n_val]
    val=set(val)
    train=[]
    for i in tqdm(range(n+1)):
        if(i not in val):
            train.append(i)
    return train,list(val)
def GETCSV(df):
    train_csv,val_csv=GetIndex(len(df))
    return df.iloc[train_csv],df.iloc[val_csv]

In [ ]:
data_transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

In [ ]:
file= pd.read_csv(PATH/'labels.csv')

In [ ]:
train_csv,val_csv=GETCSV(file)

In [ ]:
train_csv.head()

In [ ]:
class DataSet(Dataset):
   
    def __init__(self, csv_file, root_dir, n_class, transform=None):
        
        self.csv_file = csv_file
        array=[]
        for i,fn in zip(csv_file['category'],csv_file['image_name']):
            array.append((fn,i))
        self.file_label=array
        self.root_dir = root_dir
        self.classes=n_class
        self.transform = transform
        self.grey=transforms.Grayscale(num_output_channels=3)
    def __len__(self):
        return len(self.csv_file)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.file_label[idx][0])
        images = Image.open(img_name).convert('RGB')
        labels = self.file_label[idx][1]
        images = self.transform(images)        
        return images,torch.tensor(labels)

In [ ]:
a=train_data[3][0]

In [ ]:
a=to_np(a.permute(1,2,0))

In [ ]:
plt.imshow(a)

In [ ]:
train_data=DataSet(train_csv,PATH/'images/',37,transform=data_transform)

In [ ]:
valid_data=DataSet(val_csv,PATH/'images/',37,transform=data_transform)

In [ ]:
train_loader=DataLoader(train_data,batch_size=20,shuffle=True ,num_workers=0 , pin_memory=True)
valid_loader=DataLoader(valid_data,batch_size=20,shuffle=True ,num_workers=0 , pin_memory=True)

In [ ]:
train_data[103][0].shape

In [ ]:
dataloaders = {'train': train_loader,'valid':valid_loader}

In [ ]:
dataset_sizes={'train':len(train_data),'valid':len(valid_data)}

In [ ]:
class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, sz=None):
        super().__init__()
        sz = sz or (1,1)
        self.ap = nn.AdaptiveAvgPool2d(sz)
        self.mp = nn.AdaptiveMaxPool2d(sz)
    def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)
    
class Flatten(nn.Module):
    def __init__(self, keep_batch_dim=True):
        super().__init__()
        self.keep_batch_dim = keep_batch_dim

    def forward(self, x):
        if self.keep_batch_dim:
            return x.view(x.size(0), -1)
        return x.view(-1)

In [ ]:
model_conv = models.resnet34(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False
layers=[Flatten(),nn.BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
       nn.Dropout(p=0.25), nn.Linear(in_features=1024, out_features=512, bias=True),
       nn.ReLU(),nn.BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
       nn.Dropout(p=0.5),nn.Linear(in_features=512, out_features=37, bias=True),nn.LogSoftmax()]
model_conv.avgpool=AdaptiveConcatPool2d(1)
model_conv.fc=nn.Sequential(*layers)

In [ ]:
lr=0.04

In [ ]:
lr_period=len(train_loader)

In [ ]:
def one_hot_embedding(labels, num_classes):
    labels=labels.type(torch.long)
    return torch.eye(num_classes)[labels.data.cpu()]

class BCE(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.num_classes = num_classes
    
    def forward(self, preds, targets):
        t = one_hot_embedding(targets, self.num_classes+1)
        t = V(t[:,:-1].contiguous()) 
        return F.binary_cross_entropy_with_logits(preds, t,size_average=False) / self.num_classes

In [ ]:
criterion = BCE(37).cuda()
model_conv = model_conv.cuda()
optimizer = optim.SGD(model_conv.fc.parameters(), lr=lr, momentum=0.9)

In [ ]:
def sgdr(period, batch_idx):
    batch_idx = float(batch_idx)
    restart_period = period
    while batch_idx/restart_period > 1.:
        batch_idx = batch_idx - restart_period
        restart_period = restart_period * 2.

    radians = math.pi*(batch_idx/restart_period)
    return 0.5*(1.0 + math.cos(radians))

In [ ]:
def set_optimizer_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return optimizer

In [ ]:
lr_trace=[]

In [ ]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    model_conv.train()
    global optimizer
    start_batch_idx = len(train_loader)*epoch
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.cuda(), targets.cuda()
        global_step = batch_idx+start_batch_idx
        batch_lr = lr*sgdr(lr_period, global_step)
        lr_trace.append(batch_lr)
        optimizer = set_optimizer_lr(optimizer, batch_lr)
        optimizer.zero_grad()
        inputs, targets = V(inputs), V(targets)
        outputs = model_conv(inputs)
#         print(outputs.shape)
#         print(targets.shape)
        loss = criterion(outputs.cuda(), targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.data.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

        print(batch_idx, len(train_loader), 'Loss: %.3f | Acc: %.3f%% (%d/%d) | LR: %.3f'
            % (train_loss/(batch_idx+1), 100.*correct/total, correct, total, batch_lr))

In [ ]:
def train_model_sgdr(num_cycle=3):
    epoch=2**num_cycle-1
    lr_trace=[]
    for e in range(epoch):
        train(e)

In [ ]:
train_model_sgdr()

In [ ]:
torch.save(model_conv.state_dict(), PATH/'modelsgd.h5')

In [ ]:
model_conv.load_state_dict(torch.load(PATH/'modelsgd.h5'), strict=False)